<a href="https://colab.research.google.com/github/stwater20/AIS3-2024-Material/blob/main/AIS3_lab1_%E5%A8%81%E8%84%85%E5%AF%A6%E9%AB%94%E6%A8%99%E8%A8%BB%E8%88%87%E8%A8%93%E7%B7%B4%E6%8F%90%E5%8F%96%E6%A8%A1%E5%9E%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install datasets --quiet

In [8]:
from datasets import Dataset, DatasetDict, ClassLabel, Sequence, Features
import pandas as pd

In [9]:
def load_data(file_path):
  tokens = []
  ner_tags = []
  temp_tokens = []
  temp_ner_tags = []
  with open(file_path, 'r', encoding='utf-8') as file:
      for line in file:
          if len(line.split())>1:
              line = line.strip()
              token, tag = line.rsplit(' ', 1)
              temp_tokens.append(token)
              temp_ner_tags.append(tag)
          else:
              tokens.append(temp_tokens)
              ner_tags.append(temp_ner_tags)
              temp_tokens = []
              temp_ner_tags = []
  ds = Dataset.from_dict({'tokens': tokens, 'ner_tags': ner_tags})
  return ds

ds_train = load_data('DNRTI/train.txt')
ds_valid = load_data('DNRTI/valid.txt')
ds_test = load_data('DNRTI/test.txt')

In [10]:
unique_tags = set()
for dataset in [ds_train, ds_valid, ds_test]:
    unique_tags.update([tag for sublist in dataset['ner_tags'] for tag in sublist])
unique_tags = sorted(list(unique_tags))
# unique_tags

In [11]:
label2id = {label: idx for idx, label in enumerate(unique_tags)}
id2label = {idx: label for label, idx in label2id.items()}

In [13]:
def update_ner_tags(example):
    return {'ner_tags': [label2id[tag] for tag in example['ner_tags']]}

ds_train = ds_train.map(update_ner_tags)
ds_valid = ds_valid.map(update_ner_tags)
ds_test = ds_test.map(update_ner_tags)

Map:   0%|          | 0/5439 [00:00<?, ? examples/s]

Map:   0%|          | 0/668 [00:00<?, ? examples/s]

Map:   0%|          | 0/679 [00:00<?, ? examples/s]

In [17]:
raw_datasets = DatasetDict({
    'train': ds_train,
    'valid': ds_valid,
    'test': ds_test
})
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 5439
    })
    valid: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 668
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 679
    })
})

In [20]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = id2label[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

The admin@338 has largely targeted organizations involved in financial , economic and trade  policy , typically using publicly available RATs   such as Poison Ivy    , as well some non-public backdoors . 
O   B-HackOrg O   O       O        O             O        O  B-Idus    O B-Idus   O   B-Idus I-Idus O O         O     B-Tool   I-Tool    I-Tool O    O  B-Tool I-Tool O O  O    O    B-Tool     I-Tool    O 


In [21]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [22]:
tokenizer.is_fast

True

In [23]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()[:5]

['[CLS]', 'The', 'ad', '##min', '@']